# Evaluation MIOST OI: 

This notebook presents the evaluation of the SSH reconstructions based on the MIOST OI ([Ubelmann et al., 2020](https://www.essoar.org/doi/10.1002/essoar.10505014.1)) and performed for the **"2020a_SSH_mapping_NATL60" ocean data challenge**. 

Two experiments are analysed: 
    
>   A) **Experiment 1**: MIOST SSH reconstruction with **4 nadir altimeters**

>   B) **Experiment 2**: MIOST SSH reconstruction with **1 SWOT + 4 altimeters**

The evaluations are based on the methods & metrics described in the "example_data_eval.ipynb" notebook

In [ ]:
import xarray as xr
import numpy
import hvplot.xarray
import pyinterp
import dask
import warnings
import xrft
import os
import sys
import pandas as pd
import logging
warnings.filterwarnings('ignore')

##### libraries versions

In [ ]:
print('xarray', xr.__version__)
print('numpy', numpy.__version__)
print('hvplot', hvplot.__version__)
print('pyinterp', pyinterp.__version__)
print('dask', dask.__version__)
print('logging', logging.__version__)
print('xrft', xrft.__version__)

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [ ]:
sys.path.append('..')

In [ ]:
from src.mod_oi import *
from src.mod_inout import *
from src.mod_regrid import *
from src.mod_eval import *
from src.mod_plot import *

##### Read Nature run SSH for mapping evaluation

In [ ]:
if not os.path.exists('../dc_ref'):
    print('dc_ref folder not found...')
    print('download it...')
    # Get nature run (it may take several minutes depending on your connection!!!!)
    !wget https://ige-meom-opendap.univ-grenoble-alpes.fr/thredds/fileServer/meomopendap/extract/ocean-data-challenges/dc_data1/dc_ref.tar.gz
    !tar -xvf dc_ref.tar.gz --directory ../
    !rm -f dc_ref.tar.gz
    
dc_ref = xr.open_mfdataset('../dc_ref/*.nc', combine='nested', concat_dim='time', parallel=True)
dc_ref

In [ ]:
# Domain for analysis
time_min = numpy.datetime64('2012-10-22')                # domain min time
time_max = numpy.datetime64('2012-12-02')                # domain max time
lon_min = -64.975                                        # domain min lon
lon_max = -55.007                                        # domain max lon
lat_min = 33.025                                         # domain min lat
lat_max = 42.9917                                        # domain max lat

##### Select time window sample for evaluation 

In [ ]:
dc_ref_sample = dc_ref.sel(time=slice(time_min, time_max), 
                           lon=slice(lon_min, lon_max), 
                           lat=slice(lat_min, lat_max), drop=True).resample(time='1D').mean()
del dc_ref
dc_ref_sample

#### - Evalutation MIOST OI with 4 nadirs (Experience 1)

In [ ]:
# Read MIOST SSH reconstruction
!wget https://ige-meom-opendap.univ-grenoble-alpes.fr/thredds/fileServer/meomopendap/extract/ocean-data-challenges/dc_data1/dc_mapping/2020a_SSH_mapping_NATL60_MIOST_en_j1_tpn_g2.nc
ds_oi1_grid = xr.open_dataset('2020a_SSH_mapping_NATL60_MIOST_en_j1_tpn_g2.nc')
ds_oi1_grid

In [ ]:
output_directory = '../results/'
if not os.path.exists(output_directory):
    os.mkdir(output_directory)

In [ ]:
%%time
# Regrid    
ds_oi1_regrid = oi_regrid(ds_oi1_grid, dc_ref_sample)
# Eval
rmse_t_oi1, rmse_xy_oi1, leaderboard_nrmse, leaderboard_nrmse_std = rmse_based_scores(ds_oi1_regrid, dc_ref_sample)
psd_oi1, leaderboard_psds_score, leaderboard_psdt_score  = psd_based_scores(ds_oi1_regrid, dc_ref_sample)
filename_rmse_t = output_directory + 'rmse_t_miost_ssh_reconstruction_2012-10-22-2012-12-02_jason1_topex-poseidon_interleaved_envisat_geosat2.nc'
filename_rmse_xy = output_directory + 'rmse_xy_miost_ssh_reconstruction_2012-10-22-2012-12-02_jason1_topex-poseidon_interleaved_envisat_geosat2.nc'
filename_psd = output_directory + 'psd_miost_ssh_reconstruction_2012-10-22-2012-12-02_jason1_topex-poseidon_interleaved_envisat_geosat2.nc'
filename_dc_ref_sample = output_directory + 'dc_ref_2012-10-22-2012-12-02_sample.nc'
filename_oi_regrid = output_directory + 'miost_ssh_reconstruction_regridded_2012-10-22-2012-12-02_jason1_topex-poseidon_interleaved_envisat_geosat2.nc'
# Save results
# rmse_t_oi1.to_netcdf(filename_rmse_t)
# rmse_xy_oi1.to_netcdf(filename_rmse_xy)
psd_oi1.name = 'psd_score'
# psd_oi1.to_netcdf(filename_psd)
# dc_ref_sample.to_netcdf(filename_dc_ref_sample)
# ds_oi1_regrid.to_netcdf(filename_oi_regrid)
# Print leaderboard
data = [['miost 4 nadirs', 
         leaderboard_nrmse, 
         leaderboard_nrmse_std, 
         leaderboard_psds_score, 
         leaderboard_psdt_score,
        'Multiscale mapping',
        'eval_miost.ipynb']]
Leaderboard = pd.DataFrame(data, 
                           columns=['Method', 
                                    "µ(RMSE) ", 
                                    "σ(RMSE)", 
                                    'λx (degree)', 
                                    'λt (days)', 
                                   'Notes',
                                   'Reference'])
print("Summary of the leaderboard metrics:")
Leaderboard
print(Leaderboard.to_markdown())

#### - Evaluation MIOST OI with 1 swot + 4 nadirs (Experience 2)

In [ ]:
# Read MIOST SSH reconstruction
!wget https://ige-meom-opendap.univ-grenoble-alpes.fr/thredds/fileServer/meomopendap/extract/ocean-data-challenges/dc_data1/dc_mapping/2020a_SSH_mapping_NATL60_MIOST_swot_en_j1_tpn_g2.nc
ds_oi2_grid = xr.open_dataset('2020a_SSH_mapping_NATL60_MIOST_swot_en_j1_tpn_g2.nc')
ds_oi2_grid

In [ ]:
# Define outputs
output_directory = '../results/'
if not os.path.exists(output_directory):
    os.mkdir(output_directory)

In [ ]:
%%time
# Regrid    
ds_oi2_regrid = oi_regrid(ds_oi2_grid, dc_ref_sample)
# Eval
rmse_t_oi2, rmse_xy_oi2, leaderboard_nrmse, leaderboard_nrmse_std = rmse_based_scores(ds_oi2_regrid, dc_ref_sample)
psd_oi2, leaderboard_psds_score, leaderboard_psdt_score  = psd_based_scores(ds_oi2_regrid, dc_ref_sample)
filename_rmse_t = output_directory + 'rmse_t_miost_ssh_reconstruction_2012-10-22-2012-12-02_swot_jason1_topex-poseidon_interleaved_envisat_geosat2.nc'
filename_rmse_xy = output_directory + 'rmse_xy_miost_ssh_reconstruction_2012-10-22-2012-12-02_swot_jason1_topex-poseidon_interleaved_envisat_geosat2.nc'
filename_psd = output_directory + 'psd_miost_ssh_reconstruction_2012-10-22-2012-12-02_swot_jason1_topex-poseidon_interleaved_envisat_geosat2.nc'
filename_dc_ref_sample = output_directory + 'dc_ref_2012-10-22-2012-12-02_sample.nc'
filename_oi_regrid = output_directory + 'miost_ssh_reconstruction_regridded_2012-10-22-2012-12-02_swot_jason1_topex-poseidon_interleaved_envisat_geosat2.nc'
# Save results
# rmse_t_oi2.to_netcdf(filename_rmse_t)
# rmse_xy_oi2.to_netcdf(filename_rmse_xy)
psd_oi2.name = 'psd_score'
# psd_oi2.to_netcdf(filename_psd)
# dc_ref_sample.to_netcdf(filename_dc_ref_sample)
# ds_oi2_regrid.to_netcdf(filename_oi_regrid)
# Print leaderboard
data = [['miost 1 swot + 4 nadirs', 
         leaderboard_nrmse, 
         leaderboard_nrmse_std, 
         leaderboard_psds_score, 
         leaderboard_psdt_score,
        'Multiscale mapping',
        'eval_miost.ipynb']]
Leaderboard = pd.DataFrame(data, 
                           columns=['Method', 
                                    "µ(RMSE) ", 
                                    "σ(RMSE)", 
                                    'λx (degree)', 
                                    'λt (days)', 
                                   'Notes',
                                   'Reference'])
print("Summary of the leaderboard metrics:")
Leaderboard
print(Leaderboard.to_markdown())

#### - PLOT EVALUATION SCORES

In [ ]:
rmse_concat = xr.concat((rmse_t_oi1, rmse_t_oi2), dim='experiment')
rmse_concat['experiment'] = ["4 nadir", "1SWOT + 4 nadirs"]
rmse_concat.hvplot.line(x='time', y='rmse_t', by='experiment', ylim=(0, 1), cmap=['royalblue', 'lightcoral'], title='RMSE-based scores')

The figure above shows the time series of the RMSE scores for the reconstruction of SSH with 1 SWOT+ 4 nadirs and with 1 SWOT.

In [ ]:
rmse_xy_concat = xr.concat((rmse_xy_oi1, rmse_xy_oi2), dim='experiment')
rmse_xy_concat['experiment'] = ["4 nadirs", "1 SWOT + 4 nadirs"]
rmse_xy_concat.hvplot.contourf(x='lon', y='lat', levels=list(numpy.arange(0.,0.75, 0.05)), height=300, width=400, cmap='Reds', subplots=True, by='experiment', clabel='RMSE[m]')

In [ ]:
psd_concat = xr.concat((psd_oi1, psd_oi2), dim='experiment')
psd_concat['experiment'] = ["4 nadirs", "1 SWOT + 4 nadirs"]

In [ ]:
plot_psd_score_v0(psd_concat)

The PSD-based score evaluates the spatio-temporal scales resolved in mapping (green area). Resolution limits can be defined as the contour where the PSD score = 0.5, black contour in the figure (i.e. space-time scales where the reconstruction SSH error level is 2 times lower than the real SSH signal). The figure above illustrates the spatio-temporal scales solved in each experiment 4 nadirs and 4 nadirs + 1 SWOT. It shows the slight increase in resolution capability from 4 nadirs altimeters to 4 nadirs + 1 SWOT altimeters for spatial wavelengths below 2 degrees. 